## **Generamos un conjunto de tamaño similar al nuestro para probar el tiempo de entrenamiento**

In [1]:
import pandas as pd
import joblib
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
drive.mount('/content/drive')

# Ruta del archivo Parquet
file_path = '/content/drive/MyDrive/TFM/embedding_kmer3.parquet'

result_df = pd.read_parquet(file_path)


Mounted at /content/drive


In [2]:
embedding_columns4 = [col for col in result_df.columns if col != 'gene_id']

# Función para concatenar los embeddings en un solo vector
def concatenate_values(row):
    return row.values[1:].tolist()  # Ignorar la primera columna (filename) y convertir a lista

# Aplicar la función a cada fila para concatenar embeddings en un solo vector
result_df['concatenated_vector'] = result_df.apply(concatenate_values, axis=1)
result_df.drop(columns=embedding_columns4, inplace=True)


In [3]:
path_int= "/content/drive/MyDrive/genes.csv"
inter = pd.read_csv(path_int)
inter.head(5)
inter['interactant_id'] = inter['interactant_id'].astype(str)
inter['gene_id'] = inter['gene_id'].astype(str)
result_df['gene_id'] = result_df['gene_id'].astype(str)
result_df.rename(columns={'gene_id': 'gene'}, inplace=True)


In [4]:
#info gen 1
inter_emb = pd.merge(inter, result_df, left_on='gene_id', right_on='gene', how='left')
inter_emb.rename(columns={'concatenated_vector': 'gen_vector'}, inplace=True)
#info gen 2
inter_emb = pd.merge(inter_emb, result_df, left_on='interactant_id', right_on='gene', how='left')
inter_emb.rename(columns={'concatenated_vector': 'inter_vector'}, inplace=True)

inter_emb.drop(columns=['gene_x'], inplace=True)
inter_emb.drop(columns=['gene_y'], inplace=True)

inter_emb = inter_emb[['gene_id', 'gen_vector','Symbol_x','seqname_x', 'interactant_id','inter_vector','Symbol_y','seqname_y']]

In [5]:
inter_emb['gen_vector'] = inter_emb['gen_vector'].apply(lambda x: np.array(x))
inter_emb['inter_vector'] = inter_emb['inter_vector'].apply(lambda x: np.array(x))

# Multiplicar los arrays
inter_emb['result'] = inter_emb.apply(lambda row: row['gen_vector'] * row['inter_vector'], axis=1)

In [6]:
inter_emb

,gene_id,gen_vector,Symbol_x,seqname_x,interactant_id,inter_vector,Symbol_y,seqname_y,result
0,1,"[-0.30370858311653137, 0.2779957056045532, 0.8...",A1BG,chr19,2886,"[-0.35419049859046936, 0.2525642216205597, 0.8...",GRB7,chr17,"[0.10757069448024925, 0.07021176899987225, 0.7..."
1,1,"[-0.30370858311653137, 0.2779957056045532, 0.8...",A1BG,chr19,80854,"[-0.2937091886997223, 0.085811547935009, 0.492...",SETD7,chr4,"[0.0892020015482986, 0.02385524181721177, 0.41..."
2,1,"[-0.30370858311653137, 0.2779957056045532, 0.8...",A1BG,chr19,84236,"[-0.30949798226356506, 0.10648950934410095, 0....",RHBDD1,chr2,"[0.0939971936706927, 0.029603626289596008, 0.3..."
3,1,"[-0.30370858311653137, 0.2779957056045532, 0.8...",A1BG,chr19,148581,"[-0.29362666606903076, 0.08880827575922012, 0....",UBE2U,chr1,"[0.08917693871705623, 0.024688319283208138, 0...."
4,1,"[-0.30370858311653137, 0.2779957056045532, 0.8...",A1BG,chr19,5655,"[-0.23013027012348175, 0.14408709108829498, 0....",KLK10,chr19,"[0.06989253827142727, 0.040055592555598096, 0...."
...,...,...,...,...,...,...,...,...,...
838498,730755,"[-0.35040584206581116, 0.395319402217865, 0.88...",KRTAP2-3,chr17,101060226,"[-0.3380163311958313, 0.08988207578659058, 0.4...",NBPF19,chr1,"[0.11844289716467138, 0.035532128470055824, 0...."
838499,100130086,"[-0.4010367691516876, 0.35942399501800537, 0.9...",HSFX2,chrX,100506164,"[-0.39479705691337585, 0.35283350944519043, 0....",HSFX1,chrX,"[0.1583281361751352, 0.12681682954101348, 0.83..."
838500,100131303,"[-0.3474559485912323, 0.13910910487174988, 0.8...",DPEP2NB,chr16,100532736,"[-0.32195761799812317, 0.14644719660282135, 0....",MICOS10-NBL1,chr1,"[0.11186608956771149, 0.020372138430395648, 0...."
838501,100287171,"[-0.3624586760997772, 0.27248984575271606, 0.7...",WASHC1,chr9,100288778,"[-0.3617749810218811, 0.2572390139102936, 0.75...",WASH8P,chr12,"[0.13112848066721305, 0.07009501922199668, 0.5..."


In [10]:
inter_emb['result'][0]

array([ 1.07570694e-01,  7.02117690e-02,  7.42195596e-01,  3.48811535e-02,
        1.87614648e-01,  5.82856758e-01,  3.88284826e-02,  3.79635319e-02,
        4.08978641e-02, -2.13388439e-03,  5.49767005e-02,  8.07650919e-01,
       -1.13461744e-03,  4.16946683e-02,  7.08188319e-03,  3.22901342e-01,
        1.00114067e-01,  1.80332974e-01,  2.33633875e-02,  5.58028192e-03,
        9.48142906e-03,  2.39957307e-01,  1.29232007e-01,  6.34348363e-03,
        3.60905878e-02,  6.28381879e-03,  7.20764159e-02,  1.73740857e-02,
        4.31849618e-02,  1.14104019e-01,  9.26112804e-02,  1.40259925e-01,
        1.08298783e-01,  5.55977589e-01,  1.37980788e-01, -3.72663412e-03,
        9.13921964e-02,  4.15016189e-01,  9.15243935e-03,  1.63261093e+00,
        3.79440369e-01,  9.48177354e-02,  7.56172789e-02,  4.37784796e-03,
        7.13422900e-03,  2.99625145e-01,  1.70099527e-02,  4.07592539e-02,
        4.73259894e-02,  1.89825884e-03,  2.38265245e-01,  4.82009762e-02,
        9.36010850e-02,  

In [ ]:
import random

X_train, X_test = train_test_split(inter_emb, test_size=0.1, random_state=42)
df_splits = np.array_split(X_train, 100)
# Selecciona 10 muestras aleatorias de las 100 divisiones
random_splits = random.sample(df_splits, 10)

In [ ]:
# Since visualizing high-dimensional data directly is not feasible,
# the plot_decision_boundary function will not be used in this context.

# Train OSVM models on each subset and collect support vectors
support_vectors = np.vstack([
    OneClassSVM(kernel='rbf', gamma='auto', nu=0.1).fit(np.vstack(df_split['result'].values)).support_vectors_
    for df_split in random_splits
])

# Train a final OSVM model on the collected support vectors
final_model = OneClassSVM(kernel='rbf', gamma='auto', nu=0.1)
final_model.fit(support_vectors)

# Note: Visualization of decision boundaries in high-dimensional space is not straightforward
# and typically not practical. You might consider dimensionality reduction techniques
# like PCA for visualization purposes, but it's outside the direct scope of this modification.

OneClassSVM(gamma='auto', nu=0.1)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Función para calcular y mostrar métricas
def calculate_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)

    print(f"Precisión: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"Exactitud: {accuracy:.4f}")
    print("-----------------------------")


In [ ]:
X_test.shape

(83851, 9)

In [ ]:

test_data =  np.vstack(X_test['result'].values)
y_true = np.ones(83851)

preds = final_model.predict(test_data)
calculate_metrics(y_true, preds)

Precisión: 1.0000
Recall: 0.9902
F1-Score: 0.9951
Exactitud: 0.9902
-----------------------------


In [ ]:
X_test['prediccion'] = preds

In [ ]:
X_test.groupby('prediccion').size()

prediccion
-1      821
 1    83030
dtype: int64

# Probamos con interacciones negativas

In [ ]:
path_int_neg_OC = '/content/drive/MyDrive/inter_neg_OCSVM.csv'
inter_neg_modelo = pd.read_csv(path_int_neg_OC)

In [ ]:
inter_neg_modelo = inter_neg_modelo.dropna()

In [ ]:
inter_neg_modelo['interactant_id'] = inter_neg_modelo['interactant_id'].astype(int)
inter_neg_modelo['gene_id'] = inter_neg_modelo['gene_id'].astype(int)

<ipython-input-16-90a033360045>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inter_neg_modelo['interactant_id'] = inter_neg_modelo['interactant_id'].astype(int)
<ipython-input-16-90a033360045>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inter_neg_modelo['gene_id'] = inter_neg_modelo['gene_id'].astype(int)


In [ ]:
inter_neg_modelo['interactant_id'] = inter_neg_modelo['interactant_id'].astype(str)
inter_neg_modelo['gene_id'] = inter_neg_modelo['gene_id'].astype(str)

In [ ]:
#info gen 1
inter_embOC_neg = pd.merge(inter_neg_modelo, result_df, left_on='gene_id', right_on='gene', how='left')
inter_embOC_neg.rename(columns={'concatenated_vector': 'gen_vector'}, inplace=True)
#info gen 2
inter_embOC_neg = pd.merge(inter_embOC_neg, result_df, left_on='interactant_id', right_on='gene', how='left')
inter_embOC_neg.rename(columns={'concatenated_vector': 'inter_vector'}, inplace=True)

inter_embOC_neg.drop(columns=['gene_x'], inplace=True)
inter_embOC_neg.drop(columns=['gene_y'], inplace=True)

In [ ]:
inter_embOC_neg['gen_vector'] = inter_embOC_neg['gen_vector'].apply(lambda x: np.array(x))
inter_embOC_neg['inter_vector'] = inter_embOC_neg['inter_vector'].apply(lambda x: np.array(x))

# Multiplicar los arrays
inter_embOC_neg['result'] = inter_embOC_neg.apply(lambda row: row['gen_vector'] * row['inter_vector'], axis=1)

In [ ]:
df_unir = inter_emb[['gene_id', 'interactant_id', 'result']]
df_unir['clas'] = 1

df_unir_neg = inter_embOC_neg[['gene_id', 'interactant_id', 'result']]
df_unir_neg['clas'] = -1

<ipython-input-20-0594e927daf6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unir['clas'] = 1
<ipython-input-20-0594e927daf6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unir_neg['clas'] = -1


In [ ]:
pred_data =  np.vstack(inter_embOC_neg['result'].values)
preds = final_model.predict(pred_data)

inter_embOC_neg['preds3'] = preds

In [ ]:
inter_embOC_neg.groupby('preds3').size()

preds3
-1     12972
 1    763190
dtype: int64

# Pegamos los embeddings a las interacciones que vamos a meter al modelo

In [ ]:
path_int_OC = '/content/drive/MyDrive/inter_OCSVM.csv'
inter_modelo = pd.read_csv(path_int_OC)

In [ ]:
inter_modelo['interactant_id'] = inter_modelo['interactant_id'].astype(str)
inter_modelo['gene_id'] = inter_modelo['gene_id'].astype(str)

In [ ]:
#info gen 1
inter_embOC = pd.merge(inter_modelo, result_df, left_on='gene_id', right_on='gene', how='left')
inter_embOC.rename(columns={'concatenated_vector': 'gen_vector'}, inplace=True)
#info gen 2
inter_embOC = pd.merge(inter_embOC, result_df, left_on='interactant_id', right_on='gene', how='left')
inter_embOC.rename(columns={'concatenated_vector': 'inter_vector'}, inplace=True)

inter_embOC.drop(columns=['gene_x'], inplace=True)
inter_embOC.drop(columns=['gene_y'], inplace=True)

In [ ]:
inter_embOC['gen_vector'] = inter_embOC['gen_vector'].apply(lambda x: np.array(x))
inter_embOC['inter_vector'] = inter_embOC['inter_vector'].apply(lambda x: np.array(x))

# Multiplicar los arrays
inter_embOC['result'] = inter_embOC.apply(lambda row: row['gen_vector'] * row['inter_vector'], axis=1)

In [ ]:
pred_data =  np.vstack(inter_embOC['result'].values)
preds = final_model.predict(pred_data)

inter_embOC['preds3'] = preds

In [ ]:
inter_embOC.groupby('preds3').size()

preds3
-1      886
 1    48273
dtype: int64

# Vamos a hacer un dataframe único para meterlo en la regresión logística


In [ ]:
import pandas as pd

# Asegúrate de que los DataFrames tengan las mismas columnas
df_unir = inter_emb[['gene_id', 'interactant_id', 'result']].copy()
df_unir['clas'] = 1

df_unir_neg = inter_embOC_neg[['gene_id', 'interactant_id', 'result']].copy()
df_unir_neg['clas'] = -1

# Concatenar los DataFrames
df_concatenado = pd.concat([df_unir, df_unir_neg])

# Resetear el índice si es necesario
df_concatenado.reset_index(drop=True, inplace=True)

df_concatenado


,gene_id,interactant_id,result,clas
0,1,2886,"[0.10757069448024925, 0.07021176899987225, 0.7...",1
1,1,80854,"[0.0892020015482986, 0.02385524181721177, 0.41...",1
2,1,84236,"[0.0939971936706927, 0.029603626289596008, 0.3...",1
3,1,148581,"[0.08917693871705623, 0.024688319283208138, 0....",1
4,1,5655,"[0.06989253827142727, 0.040055592555598096, 0....",1
...,...,...,...,...
1614660,7756,57476,"[0.08154703447700662, 0.009399273352286297, 0....",-1
1614661,4968,55504,"[0.09896518393290421, 0.02378276617842845, 0.2...",-1
1614662,1036,151742,"[0.06384923729151826, 0.005826754388443489, 0....",-1
1614663,1002,64147,"[0.12852399924043212, 0.04426247493515367, 0.5...",-1


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Asumimos que df_concatenado ya está creado y contiene las columnas 'result' y 'clas'

# Separar características (X) y etiquetas (y)
X = np.vstack(df_concatenado['result'])
y = df_concatenado['clas']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear el modelo de regresión logística con más iteraciones y un solver diferente
model = LogisticRegression(solver='saga', max_iter=2000)

# Entrenar el modelo
model.fit(X_train_scaled, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test_scaled)

# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 0.6637888856546539
Classification Report:
               precision    recall  f1-score   support

          -1       0.66      0.62      0.64     77689
           1       0.67      0.70      0.68     83778

    accuracy                           0.66    161467
   macro avg       0.66      0.66      0.66    161467
weighted avg       0.66      0.66      0.66    161467

Confusion Matrix:
 [[48219 29470]
 [24817 58961]]


In [ ]:
import joblib

# Guardar el modelo entrenado en un archivo
model_path = '/content/drive/MyDrive/logistic_regression_model_3.joblib'
joblib.dump(model, model_path)

print(f"Modelo guardado en {model_path}")

Modelo guardado en /content/drive/MyDrive/logistic_regression_model_3.joblib


In [ ]:
pred_scaled = scaler.transform(np.vstack(inter_embOC['result'].values))
y_predscaled = model.predict(pred_scaled)
inter_embOC['preds3_logis'] = y_predscaled
inter_embOC.groupby('preds3_logis').size()

inter_embOC[['Name_x', 'Name_y', 'gene_id', 'interactant_id', 'seqname_x','seqname_y','preds3','preds3_logis']].to_csv('/content/drive/MyDrive/prediccion_3.csv', index = False)

# Miramos la sensibilidad de los casos positivos

In [ ]:
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler

# Cargar el modelo desde el archivo
model_path = '/content/drive/MyDrive/logistic_regression_model_3.joblib'
model3 = joblib.load(model_path)

# Asumimos que df_concatenado ya está creado y contiene las columnas 'result' y 'clas'

# Separar características (X) y etiquetas (y)
X = np.vstack(df_concatenado['result'])
y = df_concatenado['clas']

# Dividir los datos en conjuntos de entrenamiento y prueba, obteniendo los índices
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(X, y, df_concatenado.index, test_size=0.1, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Hacer predicciones en el conjunto de prueba
y_pred = model3.predict(X_test_scaled)

# Calcular la sensitividad para la clase 1
sensitivity = recall_score(y_test, y_pred, pos_label=1)
print("Sensitividad para la clase 1:", sensitivity)

# Crear un DataFrame con la parte de test de df_concatenado
df_test = df_concatenado.loc[test_indices].copy()
df_test['Predicted'] = y_pred

# Guardar el DataFrame en un archivo CSV si lo deseas
df_test[['gene_id','interactant_id','clas','Predicted']].to_csv('/content/drive/MyDrive/test_predictions_with_df_concatenado3.csv', index=False)


Sensitividad para la clase 1: 0.7037766478073002


# Reentrenamos el modelo con solo los 100 coeficientes más grandes

In [ ]:
t3 = [4, 549, 311,  68, 629, 656, 562, 443, 181, 545, 241, 210, 109,
       748, 107, 631, 529, 671, 347,  41,  76, 689, 528, 736,  11,  60,
       657, 608, 506, 160,  81, 652, 577, 618, 723, 482, 397,   2, 724,
        65, 255,  40, 725, 426, 459, 435, 439, 530,  79, 153, 332, 274,
       125,  74, 674, 309, 122,  37, 131, 624, 445, 227, 710, 540, 705,
       206, 198, 570, 228, 263, 193,  22, 446, 356, 755, 216, 188, 166,
       583, 717, 409, 346, 599, 261, 415, 303, 614, 742, 735, 365, 422,
       714, 766, 335, 116, 537, 333,  39, 483, 300]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Asumimos que df_concatenado ya está creado y contiene las columnas 'result' y 'clas'

# Separar características (X) y etiquetas (y)
X = np.vstack(df_concatenado['result'])
y = df_concatenado['clas']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(X, y, df_concatenado.index, test_size=0.1, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_filtered = X_train_scaled[:, t3]
X_test_filtered = X_test_scaled[:, t3]

# Crear el modelo de regresión logística con más iteraciones y un solver diferente
model = LogisticRegression(solver='saga', max_iter=2000)

# Entrenar el modelo
model.fit(X_train_filtered, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test_filtered)

# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Crear un DataFrame con la parte de test de df_concatenado
df_test = df_concatenado.loc[test_indices].copy()
df_test['Predicted'] = y_pred

# Guardar el DataFrame en un archivo CSV si lo deseas
df_test[['gene_id','interactant_id','clas','Predicted']].to_csv('/content/drive/MyDrive/test_predictions_with_df_concatenado3_nuevo.csv', index=False)

Accuracy: 0.6400998346411341
Classification Report:
               precision    recall  f1-score   support

          -1       0.64      0.58      0.61     77689
           1       0.64      0.69      0.67     83778

    accuracy                           0.64    161467
   macro avg       0.64      0.64      0.64    161467
weighted avg       0.64      0.64      0.64    161467

Confusion Matrix:
 [[45196 32493]
 [25619 58159]]


In [ ]:
import joblib

# Guardar el modelo entrenado en un archivo
model_path = '/content/drive/MyDrive/logistic_regression_model_3_nuevo100.joblib'
joblib.dump(model, model_path)

print(f"Modelo guardado en {model_path}")

Modelo guardado en /content/drive/MyDrive/logistic_regression_model_3_nuevo100.joblib
